# Transferring parameters for weighted MaxCut

The ability to transfer from unweighted to weighted graphs is well documented (see e.g. `examples/Transferability_to_unseen_instances.ipynb`). In this notebook, we demonstrate how to transfer the parameters from pre-optimized unweighted to weighted graphs.

In [1]:
import networkx as nx                    
import pandas as pd                      
import numpy as np                       
from tqdm import tqdm                    
tqdm.pandas()                            
                                         
from QAOAKit import (                
    beta_to_qaoa_format,                 
    gamma_to_qaoa_format,                
    qaoa_maxcut_energy,                  
)                  

## Step 0: load unseen graphs and optimal parameters for them

To evaluate the power of this method, we will consider some 14-node random Erdos-Renyi graphs with uniformly random edge weights between [-1,1] for which we have previously optimized the parameters. This step can be skipped if you are trying to use parameters for your own graph!

In this example, we only consider `p=2`.

In [2]:
from QAOAKit.utils import (
    get_full_weighted_qaoa_dataset_table, 
)  

p = 2

df = get_full_weighted_qaoa_dataset_table()
df = df[(df['n']==8) & (df['p_max'] == p) & (df['Weight distribution'] == 'neg_uniform')].head(50)

In [3]:
# Optional: verify that the results in the dataset are correct (this may take a few minutes)

df['Energy reproduced'] = df.progress_apply(
    lambda row: qaoa_maxcut_energy(
            row["G"],
            beta_to_qaoa_format(row["beta"]),
            gamma_to_qaoa_format(row["gamma"]),
        ), axis = 1
)

assert(np.allclose(df['Energy reproduced'], df['C_opt']))

100%|██████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:01<00:00, 31.02it/s]


# Step 1: load parameters to transfer

Following the methodology of "Parameter Transfer for Quantum Approximate Optimization of Weighted MaxCut", we transfer rescaled median (over all non-isomprphic 9-node graphs) parameters. These are pre-computed and can be loaded directly. We also load the pre-trained generative model (KDE), which we will use later.

In [4]:
from QAOAKit.parameter_optimization import get_median_pre_trained_kde

median, kde = get_median_pre_trained_kde(p) 

# Step 2: transfer

Let's start by transferring to just one graph

Recall that the formula is given by
$$\beta_{w} = \beta_{\text{median}}^S$$
$$\gamma_{w} =  \gamma_{\text{median}}^S\frac{\arctan(\frac{1}{\sqrt{d_w-1}})}{\overline{\left| w \right|}}$$
where $d_w$ is the average degree and $\overline{\left| w \right|}$ is the average edge weight of the weighted graph to which the median parameters are being transferred

In [5]:
G = df.iloc[1]["G"]
d_w = df.iloc[1]['average degree']
w = df.iloc[1]['mean(abs(weight))']
energy_with_directly_optimized_parameters = df.iloc[1]['C_opt']
energy_with_transferred_parameters = qaoa_maxcut_energy(
    G, 
    beta_to_qaoa_format(median[p:]), 
    gamma_to_qaoa_format(median[:p] * np.arctan(1/np.sqrt(d_w-1)) / w)
)
optimal_energy = df.iloc[1]['C_{true opt}']
minimal_energy = df.iloc[1]['C_{true min}']

In [6]:
print(f"Energy with transferred parameters: {energy_with_transferred_parameters}")
print(f"Energy with directly optimized parameters: {energy_with_directly_optimized_parameters}")

r_transf = (energy_with_transferred_parameters - minimal_energy) / (optimal_energy - minimal_energy)
print(f"""Approximation ratio with median parameters: {r_transf}""")
r_opt = (energy_with_directly_optimized_parameters - minimal_energy) / (optimal_energy - minimal_energy)
print(f"""Approximation ratio with optimized parameters: {r_opt}""")

Energy with transferred parameters: 6.972682143555524
Energy with directly optimized parameters: 7.1411107476
Approximation ratio with median parameters: 0.790434355548791
Approximation ratio with optimized parameters: 0.807156983677108


In [7]:
print(f"Approximation ratio gap between transferred and optimized: {(r_opt-r_transf)*100:.2f} p.p.")

Approximation ratio gap between transferred and optimized: 1.67 p.p.


The approximation ratio with transferred parameters is only 1.7 percentage point lower!

### Leveraging pre-trained generative model

We can further reduce the gap by using the pre-trained Kernel Density Estimation model. We sample 10 parameters from it in addition to the median, and pick the best result.

In [8]:
sampled_params = kde.sample(10, random_state=0)
parameters = np.vstack([np.atleast_2d(median), sampled_params])

energy_with_KDE_parameters = max([                                           
    qaoa_maxcut_energy(
        G,
        beta_to_qaoa_format(parameter[p:]),
        gamma_to_qaoa_format(parameter[:p] * np.arctan(1/np.sqrt(d_w-1))) / w
    ) for parameter in parameters
])
r_kde = (energy_with_KDE_parameters - minimal_energy) / (optimal_energy - minimal_energy)
print(f"Approximation ratio with KDE parameters: {r_kde}, gap between KDE and optimized: {(r_opt-r_kde)*100:.2f} p.p.")

Approximation ratio with KDE parameters: 0.7975563371859917, gap between KDE and optimized: 0.96 p.p.


The gap between directly optimized and transferred parameters is now only 1 percentage point!

## Did we get lucky?

Let's see if this holds for all 50 graphs in our dataset

In [9]:
df['QAOA approximatio ratio with transferred parameters'] = df.progress_apply(
    lambda row: (qaoa_maxcut_energy(
        row['G'], 
        beta_to_qaoa_format(median[p:]), 
        gamma_to_qaoa_format(median[:p]  * np.arctan(1/np.sqrt(row['average degree']-1))) / row['mean(abs(weight))']
    ) - row['C_{true min}']) / (row['C_{true opt}'] - row['C_{true min}']), axis=1
)

df[
    'QAOA approximatio ratio with directly optimized parameters'
] = (df['C_opt'] - df['C_{true min}']) / (df['C_{true opt}'] - df['C_{true min}'])

100%|██████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:01<00:00, 36.94it/s]


In [10]:
df['gap'] = df['QAOA approximatio ratio with directly optimized parameters'] - df['QAOA approximatio ratio with transferred parameters']


print(f"""The decrease in approximation ratio from using only transferred median parameters is {
    df['gap'].median() * 100:.1f} percentage points""")

The decrease in approximation ratio from using only transferred median parameters is 3.1 percentage points


The transfer works well on all our graphs!